In [ ]:
Introduction
Your task is to download, pre-process and display the free public Lending Club dataset which contains anonymised details of account activity from
LendingClub loan activity. You should use python pandas for the pre-processing and JavaScript to display the results.

Data
Lending Club Loan Data file: https://www.lendingclub.com/info/download-data.action
Pre-process
You should apply the following transforms to the data:
1. Read the csv into a Python pandas dataframe
2. Select just the grade, home_ownership, funded_amnt and funded_amnt_inv fields
3. Add a user_id column, which should be a sequence from 0 to n
4. Fill in the missing grade values with the most frequent value in that column
5. Replace missing home_ownership values with the median value for that column
6. Normalise the funded_amnt data so that it is scaled from 1-100
7. Optional, if you have time: Randomly permute the rows in the funded_amnt_inv column and interpolate the missing values (ideally using a linear
interpolation)
8. Write the data out in a format that is consumable by JavaScript


In [4]:
import pandas as pd

# 1. Read the csv into a Python pandas dataframe
# 2. Select just the grade, home_ownership, funded_amnt and funded_amnt_inv fields

data = pd.read_csv("LoanStats3a.csv", skiprows=[0],usecols=["grade", "home_ownership", "funded_amnt", "funded_amnt_inv"])
data.head()


,funded_amnt,funded_amnt_inv,grade,home_ownership
0,5000.0,4975.0,B,RENT
1,2500.0,2500.0,C,RENT
2,2400.0,2400.0,C,RENT
3,10000.0,10000.0,C,RENT
4,3000.0,3000.0,B,RENT


In [5]:
# 3. Add a user_id column, which should be a sequence from 0 to n
data.insert(0, 'user_id', range(0, len(data)))
data.head()

,user_id,funded_amnt,funded_amnt_inv,grade,home_ownership
0,0,5000.0,4975.0,B,RENT
1,1,2500.0,2500.0,C,RENT
2,2,2400.0,2400.0,C,RENT
3,3,10000.0,10000.0,C,RENT
4,4,3000.0,3000.0,B,RENT


In [6]:
#4. Fill in the missing grade values with the most frequent value in that column

# group and count the grades
grades = data.groupby(['grade']).size()

# get the index of the max value
mcv = grades.idxmax()

# fill nan with the most common value (mcv)
data["grade"] = data["grade"].fillna(mcv)

data["grade"].head()

0    B
1    C
2    C
3    C
4    B
Name: grade, dtype: object

In [7]:
# 5. Replace missing home_ownership values with the median value for that column

# group and count the home_ownership
homeOwn = data.groupby(['home_ownership']).size()

# get the median of the count and find the value
medianCount = homeOwn.median()
medianValue = 0
for val in homeOwn.index:
    if homeOwn[val]==medianCount:
        medianValue = val

# fill nan with the median value
data["home_ownership"] = data["home_ownership"].fillna(medianValue)

data["home_ownership"].head()

0    RENT
1    RENT
2    RENT
3    RENT
4    RENT
Name: home_ownership, dtype: object

In [8]:
# 6. Normalise the funded_amnt data so that it is scaled from 1-100

from sklearn import preprocessing 
import numpy as np

# fill nan with zeroes
data["funded_amnt"] = data["funded_amnt"].fillna(0)

# using MinMaxScaler convert the value to a range fro 1-100
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1, 100))
np_scaled = min_max_scaler.fit_transform(data["funded_amnt"])
data["funded_amnt"] = np_scaled

data["funded_amnt"].head()

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


0    15.142857
1     8.071429
2     7.788571
3    29.285714
4     9.485714
Name: funded_amnt, dtype: float64

In [9]:
# 7. Optional, if you have time: Randomly permute the rows in the funded_amnt_inv column and interpolate the missing values (ideally using a linear
# interpolation)

# randomly permutate the funded_amnt_inv using numpy random permutation
perm = data["funded_amnt_inv"].reindex(np.random.permutation(data["funded_amnt_inv"].index))

# interpolate the 
perm = perm.interpolate(method="linear")

data["funded_amnt_inv"]= perm


In [12]:
# 8. Write the data out in a format that is consumable by JavaScript

with open('html/temp.json', 'w') as f:
    f.write("[" + data.to_json(orient='records')[1:-1].replace('},{', '},{') + "]")